In [8]:
pip install pandas

     |████████████████████████████████| 12.6 MB 159 kB/s eta 0:00:01
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
     |████████████████████████████████| 20.6 MB 6.7 MB/s eta 0:00:011     |███████████████████████         | 14.8 MB 6.7 MB/s eta 0:00:01
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Definir las URLs y los fondos correspondientes
urls_fondos = {
    'A': 'https://www.spensiones.cl/apps/valoresCuotaFondo/vcfAFP.php?tf=A',
    'B': 'https://www.spensiones.cl/apps/valoresCuotaFondo/vcfAFP.php?tf=B',
    'C': 'https://www.spensiones.cl/apps/valoresCuotaFondo/vcfAFP.php?tf=C',
    'D': 'https://www.spensiones.cl/apps/valoresCuotaFondo/vcfAFP.php?tf=D',
    'E': 'https://www.spensiones.cl/apps/valoresCuotaFondo/vcfAFP.php?tf=E'
}

# Cuotas ingresadas manualmente con los nombres correctos de las AFP
cuotas_manual = {
    ('UNO', 'A'): '*',
    ('UNO', 'B'): '*',
    ('UNO', 'C'): '*',
    ('UNO', 'D'): '*',
    ('UNO', 'E'): '*',
    ('CAPITAL', 'A'): '*',
    ('CAPITAL', 'B'): '*',
    ('CAPITAL', 'C'): '*',
    ('CAPITAL', 'D'): '*',
    ('CAPITAL', 'E'): '*',
    ('CUPRUM', 'A'): '*',
    ('CUPRUM', 'B'): '*',
    ('CUPRUM', 'C'): '*',
    ('CUPRUM', 'D'): '*',
    ('CUPRUM', 'E'): '*',
    ('HABITAT', 'A'): '*',
    ('HABITAT', 'B'): '*',
    ('HABITAT', 'C'): '*',
    ('HABITAT', 'D'): '*',
    ('HABITAT', 'E'): '*',
    ('MODELO', 'A'): '*',
    ('MODELO', 'B'): '*',
    ('MODELO', 'C'): '*',
    ('MODELO', 'D'): '*',
    ('MODELO', 'E'): '*',
    ('PLANVITAL', 'A'): '*',
    ('PLANVITAL', 'B'): '*',
    ('PLANVITAL', 'C'): '*',
    ('PLANVITAL', 'D'): '*',
    ('PLANVITAL', 'E'): '*',
    ('PROVIDA', 'A'): '67448,00' ,
    ('PROVIDA', 'B'): '56574,56',
    ('PROVIDA', 'C'): '54462,97',
    ('PROVIDA', 'D'): '44167,25',
    ('PROVIDA', 'E'): '52609,01',
}

# Inicializar lista para almacenar los datos de todos los fondos
dataframes = []

# Variable para almacenar la última fecha disponible del web scraping
ultima_fecha = None

# Iterar sobre cada URL y fondo
for fondo, url in urls_fondos.items():
    try:
        # Realizar la solicitud GET
        response = requests.get(url, timeout=10)

        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            # Parsear el contenido HTML con BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Encontrar la fecha
            date_tag = soup.find_all('table', class_='table table-striped table-hover table-bordered table-condensed')[1]
            if date_tag:
                date_str = date_tag.find_all('center')[0].text.strip().split()[0]
                if ultima_fecha is None:
                    ultima_fecha = date_str
            else:
                date_str = "Fecha no encontrada"

            # Encontrar la tabla correcta
            table = soup.find_all('table', class_='table table-striped table-hover table-bordered table-condensed')[1]

            # Inicializar listas para almacenar los datos
            afp = []
            valor_cuota = []
            valor_patrimonio = []

            # Recorrer las filas de la tabla
            rows = table.find_all('tr')[2:]  # Ajuste para incluir todas las filas relevantes

            for row in rows:
                columns = row.find_all('td')
                if len(columns) == 3:  # Asegurarse de que la fila tiene 3 columnas
                    afp.append(columns[0].text.strip())
                    valor_cuota.append(columns[1].text.strip())
                    valor_patrimonio.append(columns[2].text.strip())

            # Crear un DataFrame con los datos
            df = pd.DataFrame({
                'A.F.P.': afp,
                'Valor Cuota': valor_cuota,
                'Valor del Patrimonio': valor_patrimonio,
                'Fecha': [date_str] * len(afp),
                'Fondo': [fondo] * len(afp)
            })

            # Agregar el DataFrame a la lista
            dataframes.append(df)

        else:
            print(f"Error al acceder a la página: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Error de conexión: {e}")

# Incorporar valores de cuota manualmente al DataFrame
afp_manual = []
valor_cuota_manual = []
valor_patrimonio_manual = []

for (afp, fondo), cuota in cuotas_manual.items():
    afp_manual.append(afp)
    valor_cuota_manual.append(cuota)
    valor_patrimonio_manual.append("")

# Crear DataFrame con los datos ingresados manualmente
if ultima_fecha is not None:
    df_manual = pd.DataFrame({
        'A.F.P.': afp_manual,
        'Valor Cuota': valor_cuota_manual,
        'Valor del Patrimonio': valor_patrimonio_manual,
        'Fecha': [ultima_fecha] * len(afp_manual),
        'Fondo': [fondo for (_, fondo) in cuotas_manual.keys()]
    })
    dataframes.insert(0, df_manual)
else:
    print("No se pudo determinar la última fecha disponible, por lo que no se pueden incluir datos manuales.")

# Concatenar todos los DataFrames en uno solo
if dataframes:
    df_consolidado = pd.concat(dataframes, ignore_index=True)

    # Reemplazar los valores de cuota (*) con valores manuales si están disponibles
    for index, row in df_consolidado.iterrows():
        if row['Valor Cuota'] == '(*)':
            manual_value = cuotas_manual.get((row['A.F.P.'], row['Fondo']), None)
            if manual_value:
                df_consolidado.at[index, 'Valor Cuota'] = manual_value

    # Priorizar los valores descargados sobre los ingresados manualmente
    df_consolidado.drop_duplicates(subset=['A.F.P.', 'Fondo'], keep='last', inplace=True)

    # Mostrar el DataFrame consolidado
    print(df_consolidado)
else:
    print("No se pudieron obtener datos de ningún fondo.")

# Verificar si se obtuvieron datos
if not df_consolidado.empty:
    # Obtener la fecha del primer valor en la columna 'Fecha'
    fecha_archivo = df_consolidado['Fecha'].iloc[0].replace("/", "-")

    # Nombre del archivo CSV con la fecha
    nombre_archivo = f"datos_fondos_{fecha_archivo}.csv"

    # Guardar el DataFrame en un archivo CSV
    df_consolidado.to_csv(nombre_archivo, index=False)

    print(f"Datos guardados en el archivo: {nombre_archivo}")
else:
    print("No se obtuvieron datos para guardar.")

       A.F.P. Valor Cuota Valor del Patrimonio          Fecha Fondo
35    CAPITAL   64.699,86    5.318.561.275.118  05-Junio-2024     A
36     CUPRUM   67.604,50    6.288.010.761.992  05-Junio-2024     A
37    HABITAT   69.062,17    7.865.227.374.849  05-Junio-2024     A
38     MODELO   66.096,78    1.866.852.984.588  05-Junio-2024     A
39  PLANVITAL   63.217,64    1.102.398.174.923  05-Junio-2024     A
40    PROVIDA   67.446,94    4.381.236.814.716  05-Junio-2024     A
41        UNO   68.593,06      303.289.013.213  05-Junio-2024     A
42    CAPITAL   57.454,56    5.855.335.025.460  05-Junio-2024     B
43     CUPRUM   59.168,36    5.728.219.098.607  05-Junio-2024     B
44    HABITAT   60.749,62    7.567.543.180.736  05-Junio-2024     B
45     MODELO   56.055,47    4.456.919.826.925  05-Junio-2024     B
46  PLANVITAL   57.313,31    2.814.562.518.739  05-Junio-2024     B
47    PROVIDA   56.573,59    5.990.665.599.557  05-Junio-2024     B
48        UNO   57.421,46      762.605.602.026  

OSError: [Errno 30] Read-only file system: 'datos_fondos_05-Junio-2024.csv'